In [68]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
------------------------------------------------------------------


In [70]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

fatal: destination path 'apex' already exists and is not an empty directory.
/content/gdrive/MyDrive/first_try_of_fastai/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-g3_waxpi
Created temporary directory: /tmp/pip-req-tracker-d4ld1bw4
Created requirements tracker '/tmp/pip-req-tracker-d4ld1bw4'
Created temporary directory: /tmp/pip-install-iw3nrvaz
Processing /content/gdrive/MyDrive/first_try_of_fastai/apex
  Created temporary directory: /tmp/pip-req-build-iiz9htra
  Added file:///content/gdrive/MyDrive/first_try_of_fastai/apex to build tracker '/tmp/pip-req-tracker-d4ld1bw4'
    Running setup.py (path:/tmp/pip-req-build-iiz9htra/setup.py) egg_info for package from file:///content/gdrive/MyDrive/first_try_of_fastai/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-req-build-iiz9htra/pip-egg-info/apex.egg-info
    writing /tmp/pip-req-build-iiz9htra/pip-egg-info/apex.egg-info/PKG-INFO
    

In [71]:
#export
import os
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai/exp')
from nb_11 import *
os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

[autoreload of apex.parallel.sync_batchnorm_kernel failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
RecursionError: maximum recursion depth exceeded
]
[autoreload of apex.normalization.fused_layer_norm failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
RecursionError: maximum recursion depth exceeded
]


In [72]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160)

In [73]:
#export
def cos_1cycle_anneal(start, high, end):
    return [sched_cos(start, high), sched_cos(high, end)]
    
def create_phases(phases):
    phases = listify(phases)
    return phases + [1-sum(phases)]
    
def noop(x): return x

class Flatten(nn.Module):
  def forward(self, x): return x.view(x.size(0), -1)

act_fn = nn.ReLU(inplace=True)

def init_cnn(m):
  if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
  if isinstance(m, (nn.Conv2d, nn.Linear)): nn.init.kaiming_normal_(m.weight)
  for l in m.children(): init_cnn(l)

def cnn_learner(arch, data, loss_func,opt_func, c_in=None, c_out=None,
                lr=1e-2, cuda=True, norm=None, progress=True, mixup=0,xtra_cb=None, **kwargs):
  cbfs = [partial(AvgStatsCallback,accuracy)] + listify(xtra_cb)
  if progress: cbfs.append(ProgressCallback)
  if cuda: cbfs.append(CudaCallback)
  if norm: cbfs.append(partial(BatchTransformXCallback, norm))
  if mixup: cbfs.append(partial(MixUp, mixup))
  arch_args = {}
  if not c_in: c_in = data.c_in
  if not c_out: c_out = data.c_out
  if c_in: arch_args['c_in'] = c_in
  if c_out: arch_args['c_out'] = c_out
  return Learner(arch(**arch_args), data, loss_func, opt_func=opt_func, lr=lr, cb_funcs=cbfs, **kwargs)

class CategoryProcessor(Processor):
    def __init__(self): self.vocab=None

    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            self.vocab = uniqueify(items)
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return self.otoi[item]

    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return self.vocab[idx]

def xresnet18 (**kwargs): return XResNet.create(1, [2,2,2 ,2], **kwargs)
def xresnet34 (**kwargs): return XResNet.create(1, [3,4,6 ,3], **kwargs)
def xresnet50 (**kwargs): return XResNet.create(4, [3,4,6 ,3], **kwargs)
def xresnet101(**kwargs): return XResNet.create(4, [3,4,23,3], **kwargs)
def xresnet152(**kwargs): return XResNet.create(4, [3,8,36,3], **kwargs)

class XResNet(nn.Sequential):
  @classmethod
  def create(cls, expansion, layers, c_in=3, c_out=1000):
    nfs = [c_in, (c_in+1)*8, 64, 64]
    stem = [conv_layer(nfs[i], nfs[i+1], stride=2 if i==8 else 1)
            for i in range(3)]
    
    nfs = [64//expansion, 64, 128, 256, 512]
    res_layers = [cls._make_layer(expansion, nfs[i], nfs[i+1],
                                 n_blocks=1, stride=1 if i==0 else 2)
                  for i,l in enumerate(layers)]

    res = cls(
        *stem,
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        *res_layers,
        nn.AdaptiveAvgPool2d(1), Flatten(),
        nn.Linear(nfs[-1]*expansion, c_out),
    )

    init_cnn(res)
    return res

  @staticmethod
  def _make_layer(expansion, ni, nf, n_blocks, stride):
      return nn.Sequential(
          *[ResBlock(expansion, ni if i== 0 else nf, nf, stride if i==0 else 1)
          for i in range(n_blocks)])

def conv(ni, nf, ks=3, stride=1, bias = False):
  return nn.Conv2d(ni, nf, kernel_size = ks, stride=stride, padding=ks//2, bias =bias)

def conv_layer(ni, nf, ks=3, stride=1, zero_bn=False, act=True):
  bn = nn.BatchNorm2d(nf)
  nn.init.constant_(bn.weight, 0. if zero_bn else 1.)
  layers = [conv(ni, nf, ks, stride=stride), bn]
  if act: layers.append(act_fn)
  return nn.Sequential(*layers)

class ResBlock(nn.Module):
  def __init__(self, expansion, ni , nh, stride = 1):
    super().__init__()
    nf,ni = nh*expansion,ni*expansion
    layers =  [conv_layer(ni, nh, 1)]
    layers += [
        conv_layer(nh, nf, 3, stride=stride, zero_bn=True, act=False)
    ] if expansion==1 else [
        conv_layer(nh, nh, 3, stride=stride),
        conv_layer(nh, nf, 1, zero_bn = True, act=False)
    ]
    self.convs = nn.Sequential(*layers)
    self.idconv = noop if ni==nf else conv_layer(ni, nf, 1, act=False)
    self.pool = noop if stride==1 else nn.AvgPool2d(2)

  def forward(self, x): return act_fn(self.convs(x) + self.idconv(self.pool(x)))

class GeneralRelu(nn.Module):
    def __init__(self, leak=None, sub=None, maxv=None):
        super().__init__()
        self.leak,self.sub,self.maxv = leak,sub,maxv

In [74]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(128, scale=(0.35, 1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(128), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y = CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers = 8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [75]:
len(il)

12954

In [76]:
loss_func = LabelSmoothingCrossEntropy()
#arch = partial(xresnet18, c_out=10)
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [77]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [78]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end = 0.95):
  phases = create_phases(pct_start)
  sched_lr = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
  sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
  return [ParamScheduler('lr', sched_lr),
          ParamScheduler('mom', sched_mom)]

In [79]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [80]:
learn.fit(40, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.162698,0.234903,2.115422,0.250191,01:23
1,2.042926,0.298615,2.028814,0.296768,01:23
2,1.968097,0.332632,1.936577,0.360143,01:23
3,1.889455,0.377729,1.923915,0.357088,01:23
4,1.841840,0.401108,1.954268,0.362433,01:23
5,1.786522,0.425042,1.880772,0.389412,01:23
6,1.750568,0.439114,2.207379,0.344871,01:23
7,1.719788,0.462161,2.125674,0.303894,01:24
8,1.693689,0.473241,1.916980,0.379486,01:23
9,1.662988,0.484100,1.682420,0.499364,01:23


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [81]:
st = learn.model.state_dict()

In [82]:
type(st)

collections.OrderedDict

In [83]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs.0.1.running_var, 5.0.convs.0.1.num_batches_tracked, 5.0.convs.1.0.weight, 5.0.convs.1.1.weight, 5.0.convs.1.1.bias, 5.0.convs.1.1.running_mean, 5.0.convs.1.1.running_var, 5.0.convs.1.1.num_batches_tracked, 5.0.idconv.0.weight, 5.0.idconv.1.weight, 5.0.idconv.1.bias, 5.0.idconv.1.running_mean, 5.0.idconv.1.

In [84]:
st['10.bias']

tensor([-0.0055,  0.0079, -0.0219, -0.0229,  0.0042,  0.0227,  0.0243,  0.0084,
        -0.0420, -0.0102], device='cuda:0')

In [85]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok = True)

In [86]:
torch.save(st, mdl_path/'iw5')

**PETS**

In [87]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [88]:
pets.ls()

[PosixPath('/root/.fastai/data/oxford-iiit-pet/images'),
 PosixPath('/root/.fastai/data/oxford-iiit-pet/annotations')]

In [89]:
pets_path = pets/'images'

In [90]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [91]:
il

ImageList (7390 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Bengal_65.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_188.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/British_Shorthair_108.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/english_setter_177.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/basset_hound_162.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Sphynx_238.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_163.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/yorkshire_terrier_84.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/pomeranian_105.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/saint_bernard_135.jpg'), ...]
Path: /root/.fastai/data/oxford-iiit-pet/images

In [92]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [93]:
random.seed(42)

In [94]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [95]:
sd

SplitData
Train: ImageList (6667 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Bengal_65.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/British_Shorthair_108.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/english_setter_177.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/basset_hound_162.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Sphynx_238.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/scottish_terrier_163.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/pomeranian_105.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/great_pyrenees_92.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/leonberger_11.jpg'), PosixPath('/root/.fastai/data/oxford-iiit-pet/images/Bombay_66.jpg'), ...]
Path: /root/.fastai/data/oxford-iiit-pet/images
Valid: ImageList (723 items)
[PosixPath('/root/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_188.jpg'), PosixPath('/root/.fastai/data/oxford-ii

In [96]:
n = il.items[0].name; n

'Bengal_65.jpg'

In [97]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'Bengal'

In [98]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [99]:
proc = CategoryProcessor()

In [100]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [101]:
', '.join(proc.vocab)

'Bengal, British_Shorthair, english_setter, basset_hound, Sphynx, scottish_terrier, pomeranian, great_pyrenees, leonberger, Bombay, american_pit_bull_terrier, Abyssinian, Ragdoll, english_cocker_spaniel, Maine_Coon, wheaten_terrier, pug, staffordshire_bull_terrier, keeshond, havanese, beagle, shiba_inu, miniature_pinscher, Persian, saint_bernard, chihuahua, boxer, Russian_Blue, Siamese, yorkshire_terrier, samoyed, Egyptian_Mau, american_bulldog, german_shorthaired, Birman, newfoundland, japanese_chin'

In [102]:
ll.valid.x.tfms = val_tfms

In [103]:
c_out = len(proc.vocab)

In [104]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [105]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [106]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.474667,0.083096,3.405414,0.088520,01:00
1,3.336200,0.121044,3.500823,0.130014,01:01
2,3.137831,0.165892,3.400853,0.141079,01:00
3,2.912923,0.233688,2.988083,0.211618,01:00
4,2.657240,0.326534,2.661012,0.319502,01:00


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


**CUSTOM HEAD**

In [107]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [108]:
st = torch.load(mdl_path/'iw5')

In [109]:
m = learn.model

In [110]:
m.load_state_dict(st)

<All keys matched successfully>

In [111]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [112]:
xb, yb = get_batch(data.valid_dl, learn)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [113]:
# added this functionality to send the model to cuda, was causing errors when model was on gpu
m_cut = m_cut.cuda()

In [114]:
pred = m_cut(xb)

In [115]:
pred.shape

torch.Size([128, 512, 8, 8])

In [116]:
ni = pred.shape[1]

In [117]:
#export
class AdaptiveConcatPool2d(nn.Module):
  def __init__(self, sz=1):
    super().__init__()
    self.output_size = sz
    self.ap = nn.AdaptiveAvgPool2d(sz)
    self.mp = nn.AdaptiveMaxPool2d(sz)
  def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [118]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out)
)

In [119]:
learn.model = m_new

In [120]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.119025,0.194690,2.518506,0.362379,01:01
1,2.428789,0.391630,2.548848,0.417704,01:01
2,2.186951,0.479826,2.147967,0.536653,01:00
3,1.916310,0.571471,1.869478,0.596127,01:01
4,1.685835,0.673616,1.692255,0.666667,01:01


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


**adapt model**

In [121]:
def adapt_model(learn, data):
  cut = next(i for i,o in enumerate(m.children()) if isinstance(o, nn.AdaptiveAvgPool2d))
  m_cut = m[:cut]
  xb, yb = get_batch(data.valid_dl, learn)
  # added this functionality to send the model to cuda, was causing errors when model was on gpu
  m_cut = m_cut.cuda()
  pred = m_cut(xb)
  ni = pred.shape[1]
  m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out))
  learn.model = m_new

In [123]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

<All keys matched successfully>

In [124]:
adapt_model(learn, data)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [125]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [126]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.407332,0.423729,2.194932,0.536653,00:38
1,2.275087,0.529324,2.630694,0.466113,00:38
2,1.790892,0.656367,1.694050,0.677732,00:38


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [127]:
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [128]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.614248,0.710664,1.729665,0.651452,00:38
1,1.646224,0.691315,1.754004,0.648686,00:38
2,1.691727,0.669267,1.733115,0.659751,00:38
3,1.602147,0.701515,1.662397,0.677732,00:38
4,1.516567,0.743813,1.635760,0.695712,00:38


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


**BATCH NORM TRANSFER** 

In [129]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [130]:
def apply_mod(m, f):
  f(m)
  for l in m.children(): apply_mod(l, f)

def set_grad(m , b):
  if isinstance(m, (nn.Linear, nn.BatchNorm2d)): return
  if hasattr(m , 'weight'):
    for p in m.parameters(): p.requires_grad_(b)

In [131]:
apply_mod(learn.model, partial(set_grad, b=False))

In [132]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.453262,0.408430,2.219124,0.518672,00:38
1,2.229344,0.538173,2.227820,0.510373,00:39
2,1.797829,0.646768,1.708756,0.670816,00:38


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [133]:
apply_mod(learn.model, partial(set_grad, b=True))

In [135]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.610731,0.695665,1.817748,0.629322,01:00
1,1.720644,0.650068,2.209824,0.514523,01:00
2,1.758590,0.640468,2.016670,0.540802,01:01
3,1.627448,0.691765,1.750434,0.629322,01:00
4,1.440496,0.773811,1.533336,0.723375,01:00


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [136]:
learn.model.apply(partial(set_grad, b = False))

Sequential(
  (0): XResNet(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): ResBlock(
        (convs): Sequential(
          (0): Sequential(
            (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
 

**DISCRIMINATIVE LR AND PARAM GROUPS**

In [137]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [138]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [139]:
def bn_splitter(m):
  def _bn_splitter(l, g1, g2):
    if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
    elif hasattr(l, 'weight'): g1 += l.parameters()
    for ll in l.children(): _bn_splitter(ll, g1, g2)

  g1,g2 = [],[]
  _bn_splitter(m[0], g1, g2)

  g2 += m[1:].parameters()
  return g1,g2

In [140]:
a,b = bn_splitter(learn.model)

In [141]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [142]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [144]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [145]:
cb_types.after_backward

'after_backward'

In [ ]:
#export
class DebugCallback(Callback):
  _order = 999
  def __init__():
  def __call__():